# MountainCar
---
>단순한 환경이지만 cartpole, acrobot 과 많이 다르다. 이 환경은 거의 풀기가 거의 불가능하다. acrobot 과 같이 성공으로부터 배우는 데, 성공하는 것이 거의 불가능하다. 카트가 언덕을 오르는 것을 보기가 너무 어렵다. 그래서 reward 를 조작해야한다.


* state : 위치, 속도 - continuous
* action : 왼쪽, 오른쪽, 가만히 놔두기
* reward : 매 프레임마다 -1, 200 프레임이 넘어가면 자연스럽게 종료
---

In [166]:
%matplotlib notebook  
import tensorflow as tf
import gym
import numpy as np
import random
import matplotlib.pyplot as plt

In [167]:
env = gym.make('MountainCar-v0')

In [168]:
size_in = env.observation_space.shape[0]
size_out = env.action_space.n
size_w1 = 6
size_w2 = 10
size_w3 = 8

lr = .1
total_episode = 1000
epsilon = 1
gamma = .95
max_pos = -.4

reward = tf.placeholder(tf.float32)
STATE_IN = tf.placeholder(tf.float32, [None, size_in])
W_1 = tf.Variable(tf.random_normal([size_in, size_w1],stddev=.01), name='W_1')
W_2 = tf.Variable(tf.random_normal([size_w1, size_w2],stddev=.01), name='W_2')
W_3 = tf.Variable(tf.random_normal([size_w2, size_w3],stddev=.01), name='W_3')
out = tf.Variable(tf.random_normal([size_w3, size_out],stddev=.01), name='W_out')

In [169]:
L_1 = tf.matmul(STATE_IN, W_1)
L_2 = tf.matmul(L_1, W_2)
L_3 = tf.matmul(L_2, W_3)
L_out = tf.sigmoid(tf.matmul(L_3, out)) ##활성화함수가 tanh - 

# simple policy gradient 학습 - likelihood x Q
loss = tf.reduce_mean(tf.log(L_out)*reward)
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [170]:
def discounted_reward(r):
    dr = np.zeros_like(r)
    sum_r = 0
    
    for i in reversed(range(0, r.size)):
        sum_r = gamma*sum_r + r[i]
        dr[i] = sum_r  
    
    return dr

In [165]:
saver = tf.train.Saver()

with tf.Session() as sess:
    
    tf.global_variables_initializer().run()
    state_buffer = []
    reward_buffer = []
    ep = 0
    state = env.reset()
    reward_sum = 0
    step = 0
    is_render = False
    mean_ep_reward = 0
    is_trained = False
    
    while not is_trained:
        done = False
        
        while not done:
            
            step = step + 1
            state = np.reshape(state, [1, size_in])
            state_buffer.append(state)

            if(random.random() < epsilon):
                action = env.action_space.sample()
            else:
                pi = sess.run(L_out, feed_dict={STATE_IN:state})
                action = np.argmax(pi)
                
            new_state, r, d, _ = env.step(action)
            
            #
            if(state[0] > -.2):
                r = 1                
            
            state = new_state
            reward_sum += r
            reward_buffer.append(r)
            
            if d:
                ep += 1
                mean_ep_reward += reward_sum

                if(ep % 10 == 0):
                    mean_ep_reward = mean_ep_reward / 10
                    print('------------ {} 평균 : {} ------'.format(ep/10,mean_ep_reward))
                    #cartpole과는 다르게 제한시간 내로 목표에 도달하지 못하면 실패한 것으로 
                    # 학습 종료

                    mean_ep_reward = 0

                if(epsilon < .2):
                    epsilon = .2
                else:
                    epsilon *= .99

                eps = np.vstack(state_buffer)
                epr = np.vstack(reward_buffer)
                epr = discounted_reward(epr)

                sess.run(train, feed_dict={STATE_IN:eps, reward:epr})                    
                print(ep, reward_sum, step)

                state_buffer, reward_buffer = [],[]                
                reward_sum = 0
                done = True
                state = env.reset()
                step = 0

    done = False

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()